In [117]:
import json
from pathlib import Path
import pandas as pd
from itertools import combinations
from collections import Counter
import networkx as nx
import colorsys
import random
import numpy as np
from node2vec import Node2Vec
import umap
import community as community_louvain
import plotly.express as px
import re
from time import sleep
import requests
from collections import defaultdict

In [118]:
# Cargar el dataset ya limpio
path = Path("../data/albums_clean.json")
with open(path, "r", encoding="utf-8") as f:
    albums = json.load(f)

print(f"✅ Álbumes cargados: {len(albums):,}")

✅ Álbumes cargados: 1,884


## 1. Matriz de co-ocurrencias básica

In [119]:
pairs = Counter()

for a in albums:
    tags = a.get("clean_tags", [])
    # eliminar duplicados dentro del álbum
    tags = sorted(set(tags))
    for combo in combinations(tags, 2):
        pairs[combo] += 1

df_pairs = (
    pd.DataFrame([(a, b, c) for (a, b), c in pairs.items()],
                 columns=["tag1", "tag2", "weight"])
    .sort_values("weight", ascending=False)
    .reset_index(drop=True)
)

print(f"🔗 Total de combinaciones únicas: {len(df_pairs):,}")
df_pairs.head(20)

🔗 Total de combinaciones únicas: 41,598


,tag1,tag2,weight
0,ambient,electronic,152
1,black metal,metal,139
2,death metal,metal,120
3,electronic,experimental,117
4,electronic,techno,116
5,electronic,house,88
6,ambient,experimental,86
7,ambient,drone,81
8,black metal,death metal,67
9,doom,metal,66


In [120]:
# Crear grafo no dirigido
G = nx.Graph()

# Agregar aristas con peso
for _, row in df_pairs.iterrows():
    G.add_edge(row["tag1"], row["tag2"], weight=row["weight"])

print(f"🕸️ Nodos: {G.number_of_nodes():,} | Enlaces: {G.number_of_edges():,}")


🕸️ Nodos: 3,898 | Enlaces: 41,598


In [121]:
degree_dict = dict(G.degree(weight="weight"))
nx.set_node_attributes(G, degree_dict, "degree")

print("🎚️ Top 10 tags por grado ponderado:")
sorted(degree_dict.items(), key=lambda x: x[1], reverse=True)[:10]

🎚️ Top 10 tags por grado ponderado:


[('electronic', 4717),
 ('metal', 2617),
 ('ambient', 2340),
 ('experimental', 2177),
 ('alternative', 1722),
 ('rock', 1680),
 ('black metal', 1206),
 ('death metal', 1128),
 ('techno', 1000),
 ('drone', 1000)]

In [122]:
graph_data = {
    "nodes": [{"id": n, "size": degree_dict[n]} for n in G.nodes()],
    "links": [
        {"source": u, "target": v, "weight": d["weight"]}
        for u, v, d in G.edges(data=True)
    ],
}

output_path = Path("../data/tag_graph_full.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(graph_data, f, ensure_ascii=False, indent=2)

print(f"💾 Grafo exportado en formato JSON: {output_path}")

💾 Grafo exportado en formato JSON: ..\data\tag_graph_full.json


In [123]:
print(f"Nodos: {len(graph_data['nodes']):,}")
print(f"Enlaces: {len(graph_data['links']):,}")

# Mostrar algunos ejemplos aleatorios
import random
print("🎵 Ejemplo de nodo:", random.choice(graph_data["nodes"]))
print("🔗 Ejemplo de enlace:", random.choice(graph_data["links"]))


Nodos: 3,898
Enlaces: 41,598
🎵 Ejemplo de nodo: {'id': 'caribbean', 'size': 26}
🔗 Ejemplo de enlace: {'source': 'rock', 'target': 'bart maris', 'weight': 1}


In [124]:

# Cargar el grafo base
path = Path("../data/tag_graph_full.json")
with open(path, "r", encoding="utf-8") as f:
    graph_data = json.load(f)

# Rango para las coordenadas
RANGE = 1000

# Generador de colores pastel
def random_color():
    h, s, l = random.random(), 0.6, 0.6
    r, g, b = colorsys.hls_to_rgb(h, l, s)
    return f"rgb({int(r*255)}, {int(g*255)}, {int(b*255)})"

# Asignar atributos iniciales
for node in graph_data["nodes"]:
    node["x"] = random.uniform(-RANGE, RANGE)
    node["y"] = random.uniform(-RANGE, RANGE)
    node["z"] = random.uniform(-RANGE, RANGE)
    node["color"] = random_color()
    node["size"] = node.get("size", 1)

# Guardar nueva versión
output_path = Path("../data/tag_graph_embedded.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(graph_data, f, ensure_ascii=False, indent=2)

print(f"💾 Grafo enriquecido exportado: {output_path}")
print(f"🌌 Nodos con posición inicial: {len(graph_data['nodes']):,}")

💾 Grafo enriquecido exportado: ..\data\tag_graph_embedded.json
🌌 Nodos con posición inicial: 3,898


## 2. Generación de modelo híbrido

En esta etapa se busca proyectar la red de co-ocurrencias de tags musicales (tag_graph_full.json) en un espacio tridimensional que preserve relaciones significativas entre géneros.
Existen tres enfoques principales para este tipo de representación:

1. Layouts de fuerza (Force-based):
Simulan el grafo como un sistema físico de partículas unidas por resortes.
Producen visualizaciones intuitivas, pero dependen de parámetros arbitrarios (repulsión, gravedad) y solo conservan relaciones locales entre nodos conectados.
Son útiles para exploraciones visuales rápidas, no para análisis estructural reproducible.

2. Reducción de dimensionalidad (UMAP, t-SNE, PCA):
Operan sobre vectores de co-ocurrencia, preservando tanto estructura local como global.
Permiten observar ejes continuos de variación musical (por ejemplo: intensidad → atmósfera → ritmo).
Su limitación es que requieren vectores de entrada previamente definidos.

3. Embeddings de red (Node2Vec, DeepWalk):
Aprenden representaciones vectoriales de cada nodo según su contexto estructural en la red, mediante recorridos aleatorios.
Capturan patrones de conectividad y comunidades, generando un espacio semántico estable y escalable, aunque sin significado geométrico directo (se proyecta luego con UMAP).

Para miNoise se adopta un enfoque híbrido:

1. Node2Vec → obtener un embedding de 128 dimensiones que codifique la estructura de la red de géneros.

2. UMAP → reducir ese espacio a tres dimensiones para su visualización interactiva, preservando cercanías locales y coherencia global.

Este pipeline combina la robustez estructural de Node2Vec con la legibilidad visual de UMAP, generando un mapa musical continuo donde los géneros próximos comparten contextos y afinidades estilísticas.

In [125]:
# Cargar el grafo de co-ocurrencias
path = Path("../data/tag_graph_full.json")
with open(path, "r", encoding="utf-8") as f:
    graph_data = json.load(f)

# Convertir a objeto de NetworkX
G = nx.readwrite.json_graph.node_link_graph(graph_data)
print(f"📦 Nodos: {G.number_of_nodes():,} | Enlaces: {G.number_of_edges():,}")

📦 Nodos: 3,898 | Enlaces: 41,598


c:\miNoise\venv\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:290: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


In [126]:
# Configuración de Node2Vec
node2vec = Node2Vec(
    G,
    dimensions=128,        # tamaño del vector embebido
    walk_length=20,        # longitud de cada recorrido aleatorio
    num_walks=200,         # cantidad de recorridos por nodo
    workers=4,             # paralelización
    seed=42
)

# Entrenamiento del modelo
model = node2vec.fit(window=10, min_count=1, batch_words=4)
print("✅ Node2Vec entrenado correctamente.")

Computing transition probabilities: 100%|██████████| 3898/3898 [00:27<00:00, 139.63it/s]


✅ Node2Vec entrenado correctamente.


In [127]:
# Extraer embeddings en el mismo orden que los nodos del grafo
nodes = list(G.nodes())
embeddings = np.array([model.wv[n] for n in nodes])

# Reducir de 128D → 3D
reducer = umap.UMAP(
    n_neighbors=15,
    min_dist=0.3,
    n_components=3,
    random_state=42
)
coords = reducer.fit_transform(embeddings)

print("✅ Reducción UMAP completada.")

c:\miNoise\venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


✅ Reducción UMAP completada.


In [128]:
# Asignar coordenadas y mantener color y tamaño previos si existían
for node, (x, y, z) in zip(nodes, coords):
    G.nodes[node]["x"] = float(x * 150)
    G.nodes[node]["y"] = float(y * 150)
    G.nodes[node]["z"] = float(z * 150)

# Convertir a formato JSON de nuevo
graph_json = nx.readwrite.json_graph.node_link_data(G)
output_path = Path("../data/tag_graph_embedded_umap.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(graph_json, f, ensure_ascii=False, indent=2)

print(f"💾 Grafo proyectado exportado en: {output_path}")

c:\miNoise\venv\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:145: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


💾 Grafo proyectado exportado en: ..\data\tag_graph_embedded_umap.json


In [129]:
# Calcular comunidades con algoritmo Louvain
partition = community_louvain.best_partition(G, weight="weight", random_state=42)

# Guardar el cluster como atributo en el grafo
nx.set_node_attributes(G, partition, "cluster")

# Visualizar conteo de comunidades detectadas
from collections import Counter
print("🎨 Comunidades detectadas:", len(set(partition.values())))
Counter(partition.values()).most_common(10)

🎨 Comunidades detectadas: 14


[(1, 755),
 (4, 681),
 (2, 577),
 (3, 506),
 (0, 373),
 (5, 252),
 (7, 226),
 (8, 214),
 (9, 149),
 (6, 114)]

In [130]:
# Generador de color pastel basado en índice de comunidad
def community_color(c):
    h = (c * 0.618033988749895) % 1.0
    s, l = 0.6, 0.55
    r, g, b = colorsys.hls_to_rgb(h, l, s)
    return f"rgb({int(r*255)}, {int(g*255)}, {int(b*255)})"

for node, data in G.nodes(data=True):
    cluster = data.get("cluster", 0)
    data["color"] = community_color(cluster)
    data["size"] = data.get("size", 1) * 1.5

In [131]:
graph_json = nx.readwrite.json_graph.node_link_data(G)
output_path = Path("../data/tag_graph_enriched.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(graph_json, f, ensure_ascii=False, indent=2)

print(f"💾 Grafo enriquecido exportado: {output_path}")

💾 Grafo enriquecido exportado: ..\data\tag_graph_enriched.json


## 3. Generar capa de Álbums

In [132]:
# === CONFIGURACIÓN ===
BASE_PATH = Path.cwd().parent / "data"
INPUT_PATH = BASE_PATH / "albums_clean.json"
OUTPUT_PATH = BASE_PATH / "tag_album_graph.json"

print("🎵 Iniciando construcción de Capa 2: Tag ↔ Álbum")
print(f"📂 Fuente: {INPUT_PATH}")

🎵 Iniciando construcción de Capa 2: Tag ↔ Álbum
📂 Fuente: c:\miNoise\miNoise_processor\data\albums_clean.json


In [134]:
with open(INPUT_PATH, "r", encoding="utf-8") as f:
    albums = json.load(f)

nodes = {}
links = []

for album in albums:
    album_id = f"album_{album['title'].replace(' ', '-').lower()}"
    artist_name = album.get("artist", {}).get("name")

    # Nodo del álbum (oculto inicialmente)
    nodes[album_id] = {
        "id": album_id,
        "type": "album",
        "title": album["title"],
        "artist": artist_name,
        "cover_url": album.get("cover_url"),
        "visible": False
    }

    for tag in album.get("clean_tags", []):
        tag_id = f"tag_{tag.lower().strip()}"

        # Nodo del tag (visible desde el inicio)
        nodes.setdefault(tag_id, {
            "id": tag_id,
            "type": "tag",
            "label": tag.lower(),
            "visible": True
        })

        links.append({"source": tag_id, "target": album_id})

graph = {"nodes": list(nodes.values()), "links": links}


In [135]:
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(graph, f, indent=2, ensure_ascii=False)

print(f"✅ Red generada con {len(nodes)} nodos y {len(links)} enlaces")
print(f"💾 Guardado en {OUTPUT_PATH}")

✅ Red generada con 5764 nodos y 15695 enlaces
💾 Guardado en c:\miNoise\miNoise_processor\data\tag_album_graph.json


In [136]:

df_nodes = pd.DataFrame(graph["nodes"])
df_links = pd.DataFrame(graph["links"])

print("🎶 Tipos de nodos:")
display(df_nodes["type"].value_counts())

print("\n🔗 Primeros 10 enlaces:")
display(df_links.head(10))

🎶 Tipos de nodos:


type
tag      3898
album    1866
Name: count, dtype: int64


🔗 Primeros 10 enlaces:


,source,target
0,tag_alternative,album_demo
1,tag_emo,album_demo
2,tag_lofi emo,album_demo
3,tag_midwest emo,album_demo
4,tag_screamo,album_demo
5,tag_skramz,album_demo
6,tag_wrocław,album_demo
7,tag_blackgaze,album_that-lasts-forever
8,tag_metal,album_that-lasts-forever
9,tag_emo,album_that-lasts-forever


## 4. Fusión de capas 

            ┌──────────────────────────────────────────┐
            │         CAPA 1: TAG → TAG (embeddings)    │
            │   Node2Vec → UMAP  → coordenadas base      │
            └──────────────────────────────────────────┘
                              ↓
                              ↓ posiciona tags
                              ↓
            ┌──────────────────────────────────────────┐
            │   CAPA 2: TAG → ÁLBUM (red bipartita)     │
            │ cada álbum obtiene coordenadas derivadas   │
            └──────────────────────────────────────────┘
                              ↓
                              ↓ unir nodos + enlaces
                              ↓
            ┌──────────────────────────────────────────┐
            │      GRAFO FINAL (para three.js)         │
            └──────────────────────────────────────────┘


In [137]:
BASE = Path.cwd().parent / "data"
ENRICH_PATH = BASE / "tag_graph_enriched.json"   # tu modelo Node2Vec+UMAP
TAG_ALBUM_PATH = BASE / "tag_album_graph.json"   # relaciones tag↔álbum
OUTPUT = BASE / "tag_album_embedded_graph_BASE.json"

with open(ENRICH_PATH, "r", encoding="utf-8") as f:
    enriched = json.load(f)
embed_nodes = enriched["nodes"] if "nodes" in enriched else enriched

with open(TAG_ALBUM_PATH, "r", encoding="utf-8") as f:
    tag_album = json.load(f)


In [138]:
# Mapear id → nodo enriquecido (con coordenadas)
embed_lookup = {n["id"]: n for n in embed_nodes}

# Confirmación opcional
list(embed_lookup.items())[:3]

[('ambient',
  {'size': 3510.0,
   'x': 1197.022032737732,
   'y': 861.2076759338379,
   'z': 877.2287607192993,
   'cluster': 0,
   'color': 'rgb(209, 71, 71)',
   'id': 'ambient'}),
 ('electronic',
  {'size': 7075.5,
   'x': 1116.6629791259766,
   'y': 945.920205116272,
   'z': 1031.6712856292725,
   'cluster': 1,
   'color': 'rgb(71, 111, 209)',
   'id': 'electronic'}),
 ('black metal',
  {'size': 1809.0,
   'x': 1253.3743858337402,
   'y': 289.6450996398926,
   'z': 1153.9804458618164,
   'cluster': 2,
   'color': 'rgb(151, 209, 71)',
   'id': 'black metal'})]

In [139]:
def normalize_tag_id(node_id: str) -> str:
    """
    Convierte IDs del grafo tag↔album al formato usado en los embeddings.
    """
    t = node_id

    # quitar prefijo
    if t.startswith("tag_"):
        t = t[4:]
    
    # reemplazar guiones por espacios
    t = t.replace("-", " ")

    # lowercase y strip: por seguridad
    return t.lower().strip()

In [140]:
print(normalize_tag_id("tag_dark-ambient"))     # → "dark ambient"
print(normalize_tag_id("tag_black-metal"))       # → "black metal"
print(normalize_tag_id("tag_ambient"))           # → "ambient"

dark ambient
black metal
ambient


In [141]:
final_nodes = []
final_links = tag_album["links"]  # los enlaces ya están listos

missing_tags = 0

In [142]:
final_nodes = []
missing_tags = []

for node in tag_album["nodes"]:
    if node["type"] == "tag":
        # convertir ID del nodo al formato de embed_lookup
        simple_id = normalize_tag_id(node["id"])

        if simple_id in embed_lookup:
            emb = embed_lookup[simple_id]
            node["x"] = emb["x"]
            node["y"] = emb["y"]
            node["z"] = emb["z"]
            node["cluster"] = emb.get("cluster")
            node["color"] = emb.get("color")
            node["size"] = emb.get("size", 1)
        else:
            missing_tags.append(simple_id)
            node["x"] = node["y"] = node["z"] = 0
            node["size"] = 1
            node["color"] = "rgb(200,200,200)"
            node["cluster"] = -1

        final_nodes.append(node)


In [143]:
album_tags = defaultdict(list)

for link in tag_album["links"]:
    src = link["source"]
    tgt = link["target"]

    if src.startswith("tag_") and tgt.startswith("album_"):
        album_tags[tgt].append(src)

In [144]:

for node in tag_album["nodes"]:
    if node["type"] == "album":
        album_id = node["id"]
        tags_for_album = album_tags.get(album_id, [])

        # reunir coords reales desde embeddings
        coords = []
        for t in tags_for_album:
            simple = normalize_tag_id(t)
            if simple in embed_lookup:
                emb = embed_lookup[simple]
                coords.append([emb["x"], emb["y"], emb["z"]])

        if coords:
            arr = np.mean(coords, axis=0)
            node["x"], node["y"], node["z"] = arr.tolist()
        else:
            node["x"] = node["y"] = node["z"] = 0

        final_nodes.append(node)

In [145]:
graph_final = {
    "nodes": final_nodes,
    "links": tag_album["links"]
}

with open(OUTPUT, "w", encoding="utf-8") as f:
    json.dump(graph_final, f, indent=2, ensure_ascii=False)

print(f"🎉 Grafo final generado con {len(final_nodes)} nodos y {len(tag_album['links'])} enlaces")
print(f"💾 Guardado en: {OUTPUT}")

print(f"Tags sin embedding: {len(missing_tags)}")

🎉 Grafo final generado con 5764 nodos y 15695 enlaces
💾 Guardado en: c:\miNoise\miNoise_processor\data\tag_album_embedded_graph_BASE.json
Tags sin embedding: 132


In [37]:
sum(1 for n in final_nodes if n["type"] == "tag")
sum(1 for n in final_nodes if n["type"] == "album")

1866

El archivo tag_album_embedded_graph.json tendrá:

- todos los tags con sus coordenadas UMAP verdaderas

- todos los álbumes posicionados geométricamente

- todos los links intactos

- estructura final lista para Three.js o cualquier motor 3D

## 5. Enriquecimiento

In [154]:
final_nodes = []

# ---- TAGS ----
for node in tag_album["nodes"]:
    if node["type"] == "tag":
        simple_id = normalize_tag_id(node["id"])

        if simple_id in embed_lookup:
            emb = embed_lookup[simple_id]
            node["x"] = emb["x"]
            node["y"] = emb["y"]
            node["z"] = emb["z"]
            node["cluster"] = emb.get("cluster")
            node["color"] = emb.get("color")
            node["size"] = emb.get("size", 1)
        else:
            node["x"] = node["y"] = node["z"] = 0
            node["size"] = 1
            node["color"] = "rgb(200,200,200)"
            node["cluster"] = -1

        final_nodes.append(node)


# ---- ALBUMS ----
for node in tag_album["nodes"]:
    if node["type"] == "album":
        album_id = node["id"]
        tags_for_album = album_tags.get(album_id, [])

        coords = []
        for t in tags_for_album:
            simple = normalize_tag_id(t)
            if simple in embed_lookup:
                emb = embed_lookup[simple]
                coords.append([emb["x"], emb["y"], emb["z"]])

        if coords:
            arr = np.mean(coords, axis=0)
            node["x"], node["y"], node["z"] = arr.tolist()
        else:
            node["x"] = node["y"] = node["z"] = 0

        final_nodes.append(node)


In [155]:
# ============================
#  FASE 2 — ENRIQUECIMIENTO
# ============================

# Cargar albums_clean
ALBUMS_CLEAN_PATH = BASE / "albums_clean.json"
OUTPUT = BASE / "tag_album_embedded_graph_FINAL.json"

with open(ALBUMS_CLEAN_PATH, "r", encoding="utf-8") as f:
    albums_clean = json.load(f)

def slugify_title(title: str) -> str:
    return title.replace(" ", "-").lower().strip()

# Crear índice: album_id → metadata completa
album_index = {}

for a in albums_clean:
    album_id = f"album_{slugify_title(a['title'])}"
    album_index[album_id] = a

def enrich_album_node(node, meta):
    ...
    return node

# === aplicar enriquecimiento ===
for n in final_nodes:
    if n["type"] == "album":
        meta = album_index.get(n["id"])
        if meta:
            enrich_album_node(n, meta)

In [156]:
graph_final = {
    "nodes": final_nodes,
    "links": final_links
}

with open(OUTPUT, "w", encoding="utf-8") as f:
    json.dump(graph_final, f, indent=2, ensure_ascii=False)

print("🎉 Grafo final exportado correctamente")
print(f"📦 Nodos: {len(final_nodes):,}")
print(f"🔗 Enlaces: {len(final_links):,}")
print(f"💾 Archivo guardado en: {OUTPUT}")

🎉 Grafo final exportado correctamente
📦 Nodos: 5,764
🔗 Enlaces: 15,695
💾 Archivo guardado en: c:\miNoise\miNoise_processor\data\tag_album_embedded_graph_FINAL.json
